In [1]:
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

import json
import requests

In [2]:
NAVER_USER_DATA_SAVED_FILE = "naver/account"

In [3]:
import pickle

naver_user_info = dict({
    "smtp_user_id": "네이버계정@naver.com",
    "smtp_user_pw": "비밀번호"
})

f = open(NAVER_USER_DATA_SAVED_FILE, 'wb')
pickle.dump(naver_user_info, f)
f.close()

In [4]:
import pickle

f = open(NAVER_USER_DATA_SAVED_FILE, 'rb')
naver_user_info = pickle.load(f)
f.close()

In [5]:
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.audio import MIMEAudio
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [6]:
import smtplib

def send_email(smtp_info, naver_user_info, msg):
    with smtplib.SMTP(smtp_info['smtp_server'], smtp_info["smtp_port"]) as server:
        server.starttls()
        
        server.login(naver_user_info['smtp_user_id'], naver_user_info['smtp_user_pw'])
        
        print(msg.as_string())
        res = server.sendmail(msg["from"], msg["to"], msg.as_string())
        
        if not res:
            print("이메일 전송 성공!")
        else:
            print(res)

In [7]:
def make_multipart(msg_dict):
    multi = MIMEMultipart(_subtype = 'mixed')
    
    for key, value in msg_dict.items():
        if key == 'image':
            with open(value['filename'], 'rb') as fp:
                msg = MIMEImage(fp.read(), _subtype = value['subtype'])
                
        msg.add_header('Content-Disposition', 'attachment', filename = os.path.basename(value['filename']))
        
        multi.attach(msg)
        
    return multi

In [9]:
import os

In [ ]:
app = Flask(__name__)
CORS(app)

@app.route('/test-email', methods=['GET'])
def test_email():
    smtp_info = dict({
        "smtp_server": "smtp.naver.com",
        "smtp_port": 587
    })
    
    msg_dict = {
        'image': {
            'maintype': 'image',
            'subtype': 'jpg',
        }
    }
    
    title = "낵아 간드앗! 가즈아!!!!!!!"
    content = "간드앗!"
    sender = naver_user_info['smtp_user_id']
    receiver = naver_user_info['smtp_user_id']
    
    msg = MIMEText(_text = content, _charset = 'utf-8')
    msg_dict['image']['filename'] = 'image/voice_phishing.jpg'
    
    multi = make_multipart(msg_dict)
    
    multi['Subject'] = title
    multi['From'] = sender
    multi['To'] = receiver
    multi.attach(msg)
    
    send_email(smtp_info, naver_user_info, multi)
    
    return jsonify("email send 완료!")

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Content-Type: multipart/mixed; boundary="===============0621411573985291076=="
MIME-Version: 1.0
Subject: =?utf-8?b?64K17JWEIOqwhOuTnOyVlyEg6rCA7KaI7JWEISEhISEhIQ==?=
From: prodak@naver.com
To: prodak@naver.com

--===============0621411573985291076==
Content-Type: image/jpg
MIME-Version: 1.0
Content-Transfer-Encoding: base64
Content-Disposition: attachment; filename="voice_phishing.jpg"

...
UAFFFFABRRRQAUUUUAFFFFAH/9k=

--===============0621411573985291076==
Content-Type: text/plain; charset="utf-8"
MIME-Version: 1.0
Content-Transfer-Encoding: base64

6rCE65Oc7JWXIQ==

--===============0621411573985291076==--



127.0.0.1 - - [06/Oct/2022 16:16:43] "GET /test-email HTTP/1.1" 200 -


이메일 전송 성공!
